In [1]:
import requests
import time
import re
import pandas as pd
import tqdm
import pymysql
from multiprocessing.dummy import Pool
addr = "0x2d50c01669949df3bd21142f8f20ab14c48d1567"
key = "ckey_2545e4a689bb4d4c87d64889755"

class GetERC20:
    def __init__(self,addr):
        #合约地址
        self.addr = addr
        #链接数据库
        self.db = pymysql.connect(host="192.168.11.22",port=3306,user="root",password="!Q2w3e4r",database="cure",charset="utf8")
        self.cursor = self.db.cursor()
        
    def save_sql(self,date,tx_hash,from_address,to_address):
        """
        写入数据库
        """
        sql = f"insert into relation2(date,tx_hash,from_address,to_address) values ('{date}','{tx_hash}','{from_address}','{to_address}')"
        self.cursor.execute(sql)
        self.db.commit()
        
    def request(self):
        """
        请求网页
        """
        # global error_url
        global res
        page_number = 1
        page_size = 1000
        while True:
            params = {
                "page-number":f"{page_number}",
                "page-size":f"{page_size}",
                "key":"ckey_2545e4a689bb4d4c87d64889755"
                }
            url = f"https://api.covalenthq.com/v1/56/address/{self.addr}/transactions_v2/?quote-currency=USD&page-size={page_size}&page-number={page_number}"
            try:
                r = requests.get(url,params=params)
            except Exception as e:
                # print(f"捕获异常{e}")
                # print("捕获异常，等待 {} 秒后继续尝试请求！".format(3))
                # time.sleep(3)
                continue
            
            if (r.status_code == 200) and (len(r.json()['data']['items'])>0):
                for i in r.json()['data']['items']:
                    try:
                        if (i['successful'] == True) and (0 < len(i['log_events']) <= 2) and (i["to_address_label"]==None):
                            date = i['block_signed_at'].replace('T',' ').replace('Z','') #交易日期
                            tx_hash = i['tx_hash'] #交易哈希
                            from_address = i['from_address'] #交易来源
                            raw_log_data = i['log_events'][-1]['raw_log_data'] #交易日志
                            to_address = '0x'+raw_log_data[-40:] #交易去处
                            res['date'].append(date)
                            res['tx_hash'].append(tx_hash)
                            res['from_address'].append(from_address)
                            res['to_address'].append(to_address)
                            self.save_sql(date,tx_hash,from_address,to_address)
                    except Exception as e:
                        continue
                page_number+=1
                age = len(res['tx_hash'])
                if age//2500>0:
                    print(f"已爬取{age}条")
            elif (r.status_code == 200) and (len(r.json()['data']['items'])==0):
                print('爬取完毕')
                self.db.close()
                break
            elif r.status_code != 200: 
                time.sleep(5)
                continue
                

    def save(self,ans):
        return ans.to_excel(f'./result/{self.addr}.xlsx',index=False)
        
        

def main():
    addr = "0xE4D3580b8A3E2AD1ef9F8C7F581817E343120984"
    res = {'date':[],'tx_hash':[],'from_address':[],'to_address':[]}
    
    ER = GetERC20(addr)
    ER.request()
    # ER.save(ans)

if __name__ == "__main__":
    main()

In [61]:
import requests
import pandas as pd
url = 'https://api.covalenthq.com/v1/56/address/0xE4D3580b8A3E2AD1ef9F8C7F581817E343120984/transfers_v2/?quote-currency=USD&format=JSON&contract-address=0x55d398326f99059fF775485246999027B3197955&page-number=0&page-size=10&key=ckey_2545e4a689bb4d4c87d64889755'
r = requests.get(url)

In [62]:
r.json()['data']['items']

[{'block_signed_at': '2022-06-04T05:23:44Z',
  'block_height': 18386783,
  'tx_hash': '0x534e7bc39ecd309705ffb6d17992ab5599e7bf44918386a4573d375946d28cbd',
  'tx_offset': 53,
  'successful': True,
  'from_address': '0x3ced129c30a3468695a26007aa16b9cedb406bdb',
  'from_address_label': None,
  'to_address': '0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984',
  'to_address_label': None,
  'value': '0',
  'value_quote': 0.0,
  'gas_offered': 1000000,
  'gas_spent': 118220,
  'gas_price': 5000000000,
  'fees_paid': None,
  'gas_quote': 0.17750964980773928,
  'gas_quote_rate': 300.3039245605469,
  'transfers': [{'block_signed_at': '2022-06-04T05:23:44Z',
    'tx_hash': '0x534e7bc39ecd309705ffb6d17992ab5599e7bf44918386a4573d375946d28cbd',
    'from_address': '0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984',
    'from_address_label': None,
    'to_address': '0xc0a70332c90d07a37dc6cc1f0cff1e85f27891d8',
    'to_address_label': None,
    'contract_decimals': 18,
    'contract_name': 'Tether USD',
    'con

In [56]:
ans = {'tx_hash':[],'symbol':[],'from':[],'to':[],'value':[]}
for i in r.json()['data']['items']:
    if (i['successful'] == True) and (0 < len(i['log_events']) <= 2) and (i['log_events'][-1]['decoded']['name']=='Transfer') :
        tx_hash = i['tx_hash']
        symbol = (i['log_events'][-1]['sender_contract_ticker_symbol'])
        from_ = (i['log_events'][-1]['decoded']['params'][0]['value'])
        to_ = (i['log_events'][-1]['decoded']['params'][1]['value'])
        value_ = (i['log_events'][-1]['decoded']['params'][2]['value'])
        ans['tx_hash'].append(tx_hash)
        ans['symbol'].append(symbol)
        ans['from'].append(from_)
        ans['to'].append(to_)
        ans['value'].append(value_)

In [58]:
pd.DataFrame(ans)

,tx_hash,symbol,from,to,value
0,0x534e7bc39ecd309705ffb6d17992ab5599e7bf449183...,USDT,0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984,0xc0a70332c90d07a37dc6cc1f0cff1e85f27891d8,254408833083807465086
1,0xaa1412bdc131cea60c7b366237cd2b8ab023943cd284...,USDT,0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984,0x0482e2269dc02c1fd31938ab9abae67622deff62,1337276734963024237
2,0x8e0be958327814c37748489b752790fd0cd6ff828a78...,ZK,0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984,0x1ab76bb346ba6ae10b594f40b451cf306f7fdc27,25250000000000000000
3,0x72bd3abc512bc73a7ad8ff39ac79788a8e96a541db78...,USDT,0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984,0x5612ab2d2dbd042a703712880e13319d3630d15a,729006653847282210505
4,0x50c3a26f081f1b0d4177a281cdd07c9c3d8f8141a81c...,USDT,0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984,0x7b7497e3f53c001336a01aa2a4a8cfa86c301012,10600000000000000000
5,0xfcbd43442d08a17086c44eea37c5698f74f27e5acaee...,ZK,0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984,0xdcbfae84e42645b63622bf0fca4058d40efae3f3,900000000000000000
6,0xfd1ef3cfe1ba5e637d792baf8841488712ff6114ca04...,USDT,0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984,0xdcbfae84e42645b63622bf0fca4058d40efae3f3,11956154991468266928
7,0x1ed9be1513fe3af68ffa2e8660bd583c1945ab1819f8...,ZK,0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984,0x1ab76bb346ba6ae10b594f40b451cf306f7fdc27,24890869512119966000
8,0xdf1031f08c72dd92abc90ac369cba2b476d2505574e3...,ZK,0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984,0xd700ccde4bc38c8c4773432721020e3d803f2a5a,450043164631999960000
9,0x288fe05dfcd2be6bff4ab095fcf0b5918e012c085329...,ZK,0xe4d3580b8a3e2ad1ef9f8c7f581817e343120984,0xd700ccde4bc38c8c4773432721020e3d803f2a5a,2000000000000000000000
